# Strategy Comparison & Ranking

Compare all implemented strategies side-by-side.

## Strategies to Compare:
1. Momentum 7d (simple)
2. Momentum 20d (simple)
3. Market Timing (mom7)
4. Market Timing (mom20)
5. RSI-based (simple)

Compare by multiple metrics and rank strategies.

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import logging

logging.basicConfig(level=logging.INFO)

print("✓ Imports complete")

## 1. Load Data

In [ ]:
from CoinTrading.data import DataLoader
from CoinTrading.indicators import Momentum, RSI, Noise, EMA

loader = DataLoader(interval='1d', start_date='2021-04-01')
tickers = loader.client.get_tickers(quote_asset='USDT')
data = loader.load_multiple(tickers[:50], skip_errors=True)

indicators = [
    Momentum(period=7), Momentum(period=20),
    RSI(period=14), Noise(period=15),
    EMA(period=7), EMA(period=20), EMA(period=50)
]

processed_data = {}
for symbol, df in data.items():
    if len(df) < 200: continue
    result = df.copy()
    for ind in indicators:
        result = ind.calculate(result)
    processed_data[symbol] = result.dropna()

print(f"Prepared {len(processed_data)} symbols")

## 2. Define Strategies

In [ ]:
from CoinTrading.strategy import MomentumSimpleStrategy, MarketTimingStrategy

strategies = {
    'Mom7 Simple': MomentumSimpleStrategy(indicator='mom7', long_top_n=5, short_bottom_n=5),
    'Mom20 Simple': MomentumSimpleStrategy(indicator='mom20', long_top_n=5, short_bottom_n=5),
    'RSI Simple': MomentumSimpleStrategy(indicator='RSI', long_top_n=5, short_bottom_n=5),
    'Mom7 Timing': MarketTimingStrategy(indicator='mom7', long_top_n=4, short_bottom_n=8, apply_noise_filter=True),
    'Mom20 Timing': MarketTimingStrategy(indicator='mom20', long_top_n=4, short_bottom_n=8, apply_noise_filter=True),
}

print(f"Testing {len(strategies)} strategies")

## 3. Run Comparison

In [ ]:
from CoinTrading.optimization import compare_strategies, generate_comparison_report, plot_strategy_comparison

# Compare all strategies
comparison_df, equity_curves = compare_strategies(
    data=processed_data,
    strategies=strategies,
    transaction_fee=0.003
)

# Print report
report = generate_comparison_report(comparison_df, title="Strategy Comparison")
print(report)

## 4. Rank Strategies

In [ ]:
from CoinTrading.optimization import rank_strategies

# Rank by multiple metrics
ranks = rank_strategies(
    comparison_df,
    metrics=['Total Return (%)', 'CAGR (%)', 'MDD (%)', 'Sharpe']
)

print("\n" + "="*80)
print("Strategy Rankings (1 = best)")
print("="*80)
print(ranks.to_string())
print(f"\n✓ Overall Winner: {ranks.index[0]}")

## 5. Equity Curve Comparison

In [ ]:
# Plot all equity curves
fig = plot_strategy_comparison(
    equity_curves,
    title='Strategy Comparison - All Strategies',
    log_scale=True,
    figsize=(15, 8)
)
plt.show()

## 6. Risk/Return Scatter

In [ ]:
# Risk vs Return scatter plot
plt.figure(figsize=(12, 8))

for name in comparison_df.index:
    plt.scatter(
        comparison_df.loc[name, 'Volatility (%)'],
        comparison_df.loc[name, 'CAGR (%)'],
        s=200,
        alpha=0.6,
        label=name
    )

plt.xlabel('Volatility (%)', fontsize=12)
plt.ylabel('CAGR (%)', fontsize=12)
plt.title('Risk vs Return', fontsize=14, fontweight='bold')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

## 7. Monthly Returns Heatmap

In [ ]:
from CoinTrading.backtesting.visualization import plot_monthly_returns

# Get best strategy
best_strategy = ranks.index[0]
best_equity = equity_curves[best_strategy]

plt.figure(figsize=(15, 8))
plt.title(f'Monthly Returns - {best_strategy}')
plot_monthly_returns(best_equity.pct_change().dropna())
plt.show()

## Summary

✅ Compared all strategies side-by-side
- Multiple performance metrics
- Risk-adjusted rankings
- Visual comparisons

**Expected Results:**
- Market timing strategies outperform simple strategies
- Mom7 with timing likely best overall
- RSI underperforms momentum-based approaches

**Key Improvements:**
- ✅ Systematic comparison framework
- ✅ Multi-metric ranking
- ✅ Proper validation methodology
- ✅ Reusable comparison utilities

## Next Steps

Ready for live trading? See the execution module:
```python
from CoinTrading.execution import Trader, OrderManager, RiskManager

# Setup (start with paper trading!)
trader = Trader(
    strategy=strategies['Mom7 Timing'],
    data_loader=loader,
    order_manager=OrderManager(paper_trading=True),
    risk_manager=RiskManager(),
    dry_run=True  # Extra safety!
)

trader.run_live()  # Monitor only
```

**Remember:** Always test thoroughly before live trading!